## Imports

In [81]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from shapely.geometry import Point
import math

## Imports

In [124]:
pd.set_option?

In [125]:
pd.set_option('display.max_columns',None)

## Links

Miami-Dade data download: https://gis-mdc.opendata.arcgis.com/datasets/MDC::property-boundary-view/about
DOR Use code descriptions: https://gis-mdc.opendata.arcgis.com/datasets/MDC::dor-code/explore

Palm Beach data download: https://opendata2-pbcgov.opendata.arcgis.com/datasets/PBCGOV::property-information-table/explore

Broward Property Appraiser "Contact Us" page: https://bcpa.net/phone.asp

## Data read-in

In [3]:
# MiamiDade = gpd.read_file('Property_Boundary_View.geojson')
PalmBeach = pd.read_csv('Property_Information_Table.csv',dtype='str')

In [176]:
Broward = gpd.read_file('broward_2022pin')

In [178]:
from pyproj import CRS

In [180]:
target_crs = CRS.from_epsg(4326)
Broward = Broward.to_crs(target_crs)

In [114]:
x = gpd.read_file('NAL16F202201.dbf')

In [182]:
y = x

In [183]:
y = y.drop(columns='geometry')

In [184]:
y['DOR_UC'] = y['DOR_UC'].astype(int)

In [185]:
y = y[(y['DOR_UC'] >= 50) & (y['DOR_UC'] <= 69)]

In [186]:
Broward['geometry'].head(1)

0    POLYGON ((-80.23083 26.33321, -80.23082 26.332...
Name: geometry, dtype: geometry

In [187]:
merged_df = pd.merge(y, Broward, left_on='PARCEL_ID', right_on='PARCELNO', how='inner')

In [188]:
columns_to_keep = ['PARCEL_ID','DOR_UC','PHY_ADDR1','PHY_CITY','geometry']

In [189]:
final_Broward = merged_df[columns_to_keep]

In [190]:
# Create a GeoDataFrame
gdf_Broward = gpd.GeoDataFrame(final_Broward, geometry='geometry')

# Save the GeoDataFrame as a GeoJSON file
gdf_Broward.to_file('BC_agri.geojson', driver='GeoJSON')

In [118]:
len(Broward)

785027

## Data Check

In [5]:
# MD_agri = MiamiDade[MiamiDade['DOR_DESC'].str.contains('AGRI',na=False)]

In [26]:
PalmBeach.PROPERTY_USE.value_counts().head(60)
PB_agri = PalmBeach[PalmBeach['PROPERTY_USE'].str.contains('AG',na=False)]
PB_agri = PB_agri[PB_agri['PROPERTY_USE'] != 'NON AG']
PB_agri = PB_agri[PB_agri['PROPERTY_USE'] != 'OPEN STORAGE']

In [27]:
PB_agri.PROPERTY_USE.value_counts()

PROPERTY_USE
AG Classification CROP SOIL CLASS 3    1777
AG Classification EQUESTRIAN           1657
AG Classification ORN/MISC AGRI         725
AG Classification GRAGSOIL CLASS 1      293
AG Classification POUL/BEES/FISH        236
AG Classification CROP SOIL CLASS 1     227
AG Classification ORCHARD GROVES        136
AG Classification EQUESTRIAN CONDO        7
Name: count, dtype: int64

## Geocode

In [72]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [73]:
PB_agri['full_address'] = PB_agri['SITE_ADDR_STR'] + ' Palm Beach County, FL'

In [74]:
def geocode(add):
    try:
        g = gmaps_key.geocode(add)
        if g:
            lat = g[0]["geometry"]["location"]["lat"]
            lng = g[0]["geometry"]["location"]["lng"]
            return (lat, lng)
    except:
        pass
    return 'No address'  # Return 'NaN' as a string

PB_agri['geocoded'] = PB_agri['full_address'].apply(geocode)

In [75]:
# Filter out rows with 'No address' in the geocoded column
PB_agri2 = PB_agri2[PB_agri2['geocoded'] != 'No address']

In [78]:
PB_agri2.columns

Index(['PARCEL_NUMBER', 'OWNER_NAME1', 'OWNER_NAME2', 'STREET_NUMBER',
       'STREET_FRACTION', 'PRE_DIR', 'STREET_NAME', 'STREET_SUFFIX_ABBR',
       'POST_DIR', 'SITE_ADDR_STR', 'MUNICIPALITY', 'NBHD', 'PADDR1', 'PADDR2',
       'PADDR3', 'CITYNAME', 'STATE', 'ZIP1', 'ZIP2', 'SALEKEY', 'CRA',
       'ACRES', 'SALE_DATE', 'BOOK', 'PAGE', 'PRICE', 'INSTRUMENT',
       'TOTAL_MARKET', 'MKT_NOT_CAPPED', 'MKT_CAPPED', 'CAP_ADJ_VAL',
       'AG_USE_VAL', 'ASSESSED_VAL', 'EXEMPTION', 'TOTAL_VALUE',
       'TOTAL_TAXABLE', 'HMSTD_FLG', 'SUBDIV_NAME', 'YEAR_ADDED',
       'LAND_MARKET', 'IMPRV_MRKT', 'PROPERTY_USE', 'MONTHS_SINCE_SALE',
       'CONFID_FLG', 'OBJECTID', 'full_address', 'geocoded'],
      dtype='object')

In [98]:
# Create a new DataFrame with necessary columns
data = PB_agri2[['PARCEL_NUMBER', 'OWNER_NAME1', 'OWNER_NAME2', 'STREET_NUMBER', 'STREET_FRACTION', 'PRE_DIR',
           'STREET_NAME', 'STREET_SUFFIX_ABBR', 'POST_DIR', 'SITE_ADDR_STR', 'SUBDIV_NAME', 'YEAR_ADDED',
           'LAND_MARKET', 'IMPRV_MRKT', 'PROPERTY_USE', 'MONTHS_SINCE_SALE', 'CONFID_FLG', 'full_address',
           'geocoded']]

# Drop rows with NaN values in the "geocoded" column
data = data.dropna(subset=['geocoded'])

# Preprocess the string representation of tuples
data['geocoded'] = data['geocoded'].str.replace('(', '').str.replace(')', '').str.split(', ')

# Convert the string representation of tuples to actual tuples
data['geocoded'] = data['geocoded'].apply(lambda x: tuple(float(coord) for coord in x) if isinstance(x, list) else None)

# Convert tuples to Point objects in the "geometry" column
data['geometry'] = data['geocoded'].apply(lambda x: Point(x[1], x[0]) if isinstance(x, tuple) and len(x) == 2 else None)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='geometry')

In [108]:
gdf = gdf.drop(columns=['geocoded'])

In [109]:
# Save the GeoDataFrame as a GeoJSON file
gdf.to_file('PB_agri.geojson', driver='GeoJSON')

In [ ]:
# MD_agri[['X_COORD','Y_COORD']] = MD_agri[['X_COORD','Y_COORD']].astype(float)
# MD_agri['X_COORD'] = MD_agri['X_COORD'] * -1

In [10]:
# MD_agri.to_file('MD_agri.geojson', driver='GeoJSON')

## Map URL Snagger

In [11]:
# base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [110]:
# cwd = os.getcwd()

# cwd = cwd.split('/')

# final_name = base_name + cwd[-1]
# print(final_name)